In [1]:
import pandas as pd
import os.path

## Exercise Notebook 4, pandas

In this exercise notebook you will have the opportunity to load the MovieLens database and perform additional analysis.

First let's load the data into a Pandas Dataframe:

In [2]:
# set here the relative path to the movielens folder
MOVIELENS="../movielens"

In [3]:
movies = pd.read_csv(os.path.join(MOVIELENS, 'movies.csv'), sep=',')
ratings = pd.read_csv(os.path.join(MOVIELENS, 'ratings.csv'), sep=',')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Movies have just one column `movieId` that is a natural way of uniquely identifying each row, when that is the case, it is useful to turn that into an index.

In [5]:
movies = movies.set_index("movieId")

First we want to convert the timestamp into a datetime object:

In [6]:
ratings['parsed_time'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [7]:
ratings.head()

,userId,movieId,rating,timestamp,parsed_time
0,1,2,3.5,1112486027,2005-04-02 23:53:47
1,1,29,3.5,1112484676,2005-04-02 23:31:16
2,1,32,3.5,1112484819,2005-04-02 23:33:39
3,1,47,3.5,1112484727,2005-04-02 23:32:07
4,1,50,3.5,1112484580,2005-04-02 23:29:40


Ratings instead do not have a row identifier, both `userId` and `movieId` reference records in other dataframes, therefore there is no good candidate for an index so we can just leave the default integer indexing.

## Year with maximum standard deviation in the rating

First assignment is to find which year has the maximum standard deviation in the rating, **not** the maximum value of the standard deviation, but the year when it occurs.
Look at the `pandas` documentation or google to find the `pandas` functions you need to use.

In [8]:
def find_year_with_max_std(ratings):
    """Function to find the year with the larger standard deviation in rating"""
    ### BEGIN SOLUTION
    return ratings.rating.groupby(ratings.parsed_time.dt.year).std().idxmax()
    ### END SOLUTION

In [9]:
assert find_year_with_max_std(ratings) == 1998, "Wrong year identified, try again!"

In [10]:
import ex04_1_pandas_test
ex04_1_pandas_test.test_year_max_std(find_year_with_max_std, ratings)

## Identify popular movies

In the rest of the notebook, we would like only to consider movies that have a significant number of ratings.

In [11]:
number_of_ratings = ratings.movieId.value_counts()

In [12]:
number_of_ratings.head()

296    67310
356    66172
318    63366
593    63299
480    59715
Name: movieId, dtype: int64

In [13]:
popular_movies = movies.reindex(number_of_ratings[number_of_ratings >= 100])

In [14]:
assert len(popular_movies) == 8546, "Wrong! check that movies with 100 ratings are accepted"

## Data cleaning

Everytime we perform a reindexing operation, `pandas` will create a row for every value of the new index, even if it doesn't exist in the original data structure, and it will mark those data as missing with `NaN` (Not A Number).

Always check if reindexing generated invalid data:

In [15]:
popular_movies.isnull().sum()

title     699
genres    699
dtype: int64

In [16]:
### BEGIN SOLUTION
popular_movies = popular_movies.dropna()
### END SOLUTION

In [17]:
assert len(popular_movies) == 7847, "Wrong! check the documentation of the function you used"

## Filter by genre

Let's implement a general function that filters movies by genres:

In [18]:
def filter_by_genre(input_movies, genre):
    """Return only movies of a specific genre"""
    ### BEGIN SOLUTION
    return input_movies[input_movies.genres.str.contains(genre)]
    ### END SOLUTION

In [19]:
fantasy_movies = filter_by_genre(popular_movies, "Fantasy")

In [20]:
assert len(fantasy_movies) == 382, """Wrong! Make sure you are filtering the popular movies"""

## Join movies and ratings

Let's create a single `DataFrame` that contains both titles and mean ratings of the popular fantasy movies.

Titles are only available in the `movies` `DataFrame`, while ratings in the `ratings` `DataFrame`, we would like to create a single DataFrame that includes Title.

create the `mean_ratings` variable by computing the mean rating for each movie:

In [21]:
### BEGIN SOLUTION
mean_ratings = ratings.rating.groupby(ratings.movieId).mean()
### END SOLUTION

In this case we don't even need to use a join operation, we can just create a new column in the `fantasy_movies` DataFrame. This will automatically match the index of `mean_ratings` with the index of `fantasy_movies` and attach to each movie its rating. Ratings for movies that are not in the `fantasy_movies` DataFrame are discarded.

In [22]:
fantasy_movies_with_ratings = fantasy_movies.assign(rating = mean_ratings)

In [23]:
assert fantasy_movies_with_ratings.loc[7842].title.startswith("Dune"), "Missing or wrong title!"

In [24]:
assert abs(fantasy_movies_with_ratings.loc[7842].rating - 3.56) < 0.01 , "Missing or wrong rating!"

## Find the highest rated fantasy movie

Again we need to find the index where a column is max, in this case rating:

In [25]:
index_of_max_rating = fantasy_movies_with_ratings.rating.idxmax()

In [26]:
highest_rated_fantasy_movie = fantasy_movies_with_ratings.loc[index_of_max_rating]

In [27]:
assert highest_rated_fantasy_movie.title.startswith("Princess"), "Nope, wrong movie!"